<a href="https://colab.research.google.com/github/s10bhavesh/monte-carlo-simulation-wireless-channel-generation-IRS-PyTorch/blob/main/DataGen_CE_SL_MonteCarlo_exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset

In [ ]:
class ChannelEstimationDataset(Dataset):
  """
  Custom Channel Estimation Dataset Generation.
  Change the arguments to return H or LS_H
  """
  pathloss_usr_irs = torch.tensor([(0.18 ** -2) / 31.6])
  pathloss_irs_bs = torch.tensor([1 / 31.6])     # 1 / 31.6
  pathloss_usr_bs = torch.tensor([1 / 33.0])
  K_dB = 10.0
  K = torch.tensor([10 ** (K_dB/10.0)])

  def __init__(self,
              num_samples,         # DataLength
              num_ris_elements,    # N
              num_antennas,        # M
              SNRdB,
              noise_var,
              output_type='H'
              ):
    """
    Arguments:
      num_samples (int): Number of data points to be generated (eg, 10000, 100000)
      num_ris_elements (int): Number of RIS elements (typically 32, 64, 256, etc.)
      num_antennas (int): Number of antennas at the base station (typically 4, 6, 8, 10, etc.)
      SNRdB (float):
      noise_var (float):
      output_type (string): 'H' for channel estimate, 'LS' for its Least Square estimate.
                            Default is set for 'H'
    Returns:
      Channel Estimates or Least-Square Estiamtes of the Channel Estimates
    """
    self.num_samples = num_samples
    self.num_ris_elements = num_ris_elements
    self.num_antennas = num_antennas
    self.num_reflection_patterns = num_ris_elements + 1     # C = N + 1,
    self.SNRdB = torch.tensor([SNRdB])
    self.noise_var = torch.tensor([noise_var])
    self.transmission_power = torch.pow(torch.tensor([10.]), torch.tensor([SNRdB/10.])) * self.noise_var
    self.output_type = output_type

  def __len__(self):
    return self.num_samples

  def __getitem__(self, index):
    # Compute S [?]
    S = torch.sqrt(self.transmission_power) * self._fft_matrix(self.num_reflection_patterns)
    S = S.to(torch.cfloat)
    W = torch.sqrt(self.noise_var) * torch.rsqrt(torch.tensor([2.0])) * \
        torch.randn(self.num_antennas, self.num_reflection_patterns, dtype=torch.cfloat)
    # Total channel
    H = self._compute_channel()
    # H = H.to(torch.cfloat)

    # Received signal at BS: X = H*S + W
    X = torch.matmul(H, S) + W.to(torch.cfloat)

    # LS Detection
    h_est_ls = self._ls_detection(S, X)

    if self.output_type == 'LS':
      return torch.stack([torch.real(h_est_ls), torch.imag(h_est_ls)], axis=0)
    elif self.output_type == 'H':
      return torch.stack([torch.real(H), torch.imag(H)], axis=0)

  def _fft_matrix(self, num_reflection_patterns):
    num_columns = torch.tensor([num_reflection_patterns])
    mult_vector = torch.arange(num_reflection_patterns, dtype=torch.float32)

    # FFT of reflection pattern
    Wn = torch.exp(1j * torch.tensor([2.0]) * torch.pi / torch.tensor([num_reflection_patterns]))
    s = torch.empty((num_reflection_patterns, num_columns), dtype=torch.cfloat)

    # Generating the FFT matrix for the refelction pattern
    for row in range(0, num_columns):
      P_row = torch.pow(Wn, row * mult_vector)
      P_row = P_row.unsqueeze(dim=0)
      s[row] = P_row
    return s

  def _compute_channel(self):
    f = torch.sqrt(self.pathloss_usr_irs) * torch.rsqrt(torch.tensor([2])) * torch.randn(self.num_ris_elements, 1, dtype=torch.cfloat)
    G = torch.sqrt(self.pathloss_irs_bs) * (torch.sqrt(self.K / (self.K+1)) + torch.sqrt(1 / (self.K+1))) * \
        torch.rsqrt(torch.tensor([2])) * torch.randn(self.num_antennas, self.num_ris_elements, dtype=torch.cfloat)
    B = torch.matmul(G, torch.diag(torch.squeeze(f, dim=-1)))
    d = torch.sqrt(self.pathloss_usr_bs) * torch.rsqrt(torch.tensor([2])) * torch.randn(self.num_antennas, 1, dtype=torch.cfloat)
    H = torch.cat((d, B), axis=1)
    return H.to(torch.cfloat)

  def _ls_detection(self, S, X):
    w_ls = torch.t(S) / torch.matmul(S, torch.t(S))
    h_est_ls = torch.matmul(X, w_ls.to(torch.cfloat))
    return h_est_ls

  def _compute_ls_estimate(self):
    pass



In [ ]:
# Example usage
NUM_SAMPLES = 10000
NUM_RIS_ELEMENTS = 64
NUM_ANTENNAS = 8
SNR_DB = 5.0
NOISE_VAR = 1.0
OUTPUT_TYPE = 'H'

dataset = ChannelEstimationDataset(num_samples=NUM_SAMPLES,
                                   num_ris_elements=NUM_RIS_ELEMENTS,
                                   num_antennas=NUM_ANTENNAS,
                                   SNRdB=SNR_DB,
                                   noise_var=NOISE_VAR,
                                   output_type=OUTPUT_TYPE)

In [ ]:
dataset = dataset
BATCH_SIZE = 512

dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

for batch in dataloader:
  # Process each batch of normalized channel responses
  print(batch.shape)


torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([512, 2, 8, 65])
torch.Size([272, 2, 8, 65])


In [ ]:
print(len(dataset))

10000


In [ ]:
print(len(dataset))
print(dataset[0][:][0])

10000
tensor([[ 2.1043e-02, -1.3161e-02, -2.2504e-02, -5.0742e-02,  2.2448e-02,
          1.8267e-02,  5.8031e-02, -4.7280e-02, -3.4597e-02,  5.7941e-02,
         -4.5244e-02,  4.7784e-02,  2.7454e-02, -1.3235e-02, -1.2333e-02,
          1.1842e-01,  9.3338e-02,  3.3490e-02,  1.3414e-01,  1.7935e-02,
         -4.1358e-05, -2.4341e-02, -1.3955e-01, -1.9769e-02,  1.3597e-02,
         -7.2258e-02,  1.7419e-01, -1.8502e-01,  1.1485e-01,  2.3398e-02,
         -4.4295e-02, -8.7002e-03,  1.1780e-01,  1.1332e-01, -2.1827e-02,
         -3.8857e-02,  3.7754e-02,  3.2458e-03, -3.0179e-02,  1.1098e-01,
          1.5506e-01, -6.9736e-02,  5.9903e-02,  3.1163e-02,  1.3771e-02,
         -2.8598e-03, -1.5413e-02, -4.0600e-03, -5.0611e-02,  2.1795e-02,
         -2.8748e-02, -9.6164e-02,  5.5520e-02, -1.1086e-01, -1.1488e-01,
         -5.3231e-02, -8.9123e-04,  5.1854e-02, -8.5202e-03,  1.4758e-02,
          5.3571e-02, -8.8515e-03, -2.1759e-01, -8.5039e-02,  1.7325e-01],
        [ 5.6421e-02, -4.6710e-

In [ ]:

num_data_pts = 100
t = torch.empty((num_data_pts, 30, 50), dtype=torch.cfloat)
# t
for num in range(num_data_pts):
  t[num] = torch.randn((30, 50), dtype=torch.cfloat)

t.shape
torch.save(t, 'test_tensor.pt')


In [ ]:
import numpy as np

num_reflection_patterns = 33
num_columns = 33
# mult_vector = np.array(list(range(0, num_reflection_patterns)))
mult_vector = torch.arange(num_reflection_patterns, dtype=torch.float32)

# FFT of reflection pattern
# Wn = np.exp( 1j * 2 * np.pi / num_reflection_patterns)
Wn = torch.exp(1j * 2.0 * torch.pi / torch.tensor([num_reflection_patterns]))
s = torch.empty((num_reflection_patterns, num_columns), dtype=torch.cfloat)

# Generating the FFT matrix for the refelction pattern
for itr in range(0, num_columns):
  # element-wise power
  # P_row = np.power(Wn, itr * mult_vector)
  P_row = torch.pow(Wn, itr * mult_vector)
  P_row = P_row.unsqueeze(dim=0)
  # print(P_row.shape)
  # s.append(P_row)
  # torch.cat((s, P_row), dim=0)
  s[itr] = P_row

# s = np.asarray(s)
s.shape

torch.Size([33, 33])

In [ ]:
s[2]

tensor([ 1.0000+0.0000j,  0.9284+0.3717j,  0.7237+0.6901j,  0.4154+0.9096j,
         0.0476+0.9989j, -0.3271+0.9450j, -0.6549+0.7557j, -0.8888+0.4582j,
        -0.9955+0.0951j, -0.9595-0.2817j, -0.7861-0.6182j, -0.5000-0.8660j,
        -0.1423-0.9898j,  0.2358-0.9718j,  0.5801-0.8146j,  0.8413-0.5406j,
         0.9819-0.1893j,  0.9819+0.1893j,  0.8413+0.5406j,  0.5801+0.8146j,
         0.2358+0.9718j, -0.1423+0.9898j, -0.5000+0.8660j, -0.7861+0.6182j,
        -0.9595+0.2817j, -0.9955-0.0951j, -0.8888-0.4582j, -0.6549-0.7558j,
        -0.3271-0.9450j,  0.0476-0.9989j,  0.4154-0.9096j,  0.7237-0.6901j,
         0.9284-0.3717j])

In [ ]:
x = torch.rand(3,3)
x, len(x)

(tensor([[0.8368, 0.8368, 0.5101],
         [0.3732, 0.4125, 0.7695],
         [0.0943, 0.4586, 0.4892]]),
 3)

In [ ]:
for i in range(len(x)):
  x[i] = torch.tensor([0,0,0])
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
s[0][0].shape

torch.Size([])

In [ ]:
c = torch.rand(33)
# c = c.squeeze(dim=-1)
c = c.unsqueeze(dim=0)
c.shape

torch.Size([1, 33])

In [ ]:
s.shape

(33, 33)

In [ ]:
import torch
a = torch.empty((33,33), dtype=torch.cfloat)
b = torch.arange(10, dtype=torch.float32)
b * 5

tensor([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45.])

In [ ]:
a.shape

torch.Size([33, 33])

In [ ]:
pathloss_usr_irs = torch.tensor([(0.18 ** -2) / 31.6])
pathloss_irs_bs = torch.tensor([1 / 31.6])     # 1 / 31.6
pathloss_usr_bs = torch.tensor([1 / 33.0])
K_dB = 10.0
K = torch.tensor([10 ** (K_dB/10.0)])


In [ ]:
K.dtype


torch.float32

In [ ]:
x = torch.randn(10,1, dtype=torch.complex64)
x.shape, x.squeeze(1).shape

(torch.Size([10, 1]), torch.Size([10]))

In [ ]:
torch.diag(x.squeeze(-1))

tensor([[-0.9613-1.1728j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.4961-0.7250j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0857+1.1686j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.7521-0.3880j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.9735-0.3334j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j,  0.0000+0.0000j],
        [ 

In [ ]:
f = torch.sqrt(pathloss_usr_irs) * torch.rsqrt(torch.tensor([2])) * torch.randn(32, 1, dtype=torch.complex64)
G = torch.sqrt(pathloss_irs_bs) * (torch.sqrt(K / (K+1)) + torch.sqrt(1 / (K+1))) * \
    torch.rsqrt(torch.tensor([2])) * torch.randn(8, 32, dtype=torch.complex64)
B = torch.matmul(G, torch.diag(torch.squeeze(f, dim=-1)))
d = torch.sqrt(pathloss_usr_bs) * torch.rsqrt(torch.tensor([2])) * torch.randn(8, 1, dtype=torch.complex64)
H = torch.cat((d, B), axis=1)

In [ ]:
H.shape

torch.Size([8, 33])

In [ ]:
d = torch.sqrt(pathloss_usr_bs) * torch.rsqrt(torch.tensor([2])) * torch.randn(8, 1, dtype=torch.complex64)
d.shape

torch.Size([8, 1])

In [ ]:
a = torch.randn(3,4)
b = torch.randn(3,1)
a,b


(tensor([[-0.0687, -0.4956, -1.1597, -0.0276],
         [-0.0793,  1.1542, -1.4126,  0.4731],
         [-0.7441,  0.4512, -1.8254,  1.2703]]),
 tensor([[ 0.5326],
         [-0.3856],
         [-0.0242]]))

In [ ]:
c = torch.cat((a,b), axis=1)
c, c.shape

(tensor([[-0.0687, -0.4956, -1.1597, -0.0276,  0.5326],
         [-0.0793,  1.1542, -1.4126,  0.4731, -0.3856],
         [-0.7441,  0.4512, -1.8254,  1.2703, -0.0242]]),
 torch.Size([3, 5]))

In [ ]:
d = torch.randn(2,3)
e = torch.randn(3,3)
d.shape, e.shape
torch.cat((d,e), dim=0)

tensor([[ 0.1400,  1.3665, -0.9510],
        [-0.5457, -1.0288, -1.0474],
        [-0.4574,  0.5549, -0.1880],
        [ 0.1661, -0.4140, -0.2604],
        [-0.6592, -1.0276,  1.5608]])